In [158]:
import pickle

In [159]:
rets = {}

In [160]:
path = "anymal_heading_pd_ret"
with open("{}.pkl".format(path), "rb+") as file:
    data = pickle.load(file)

In [161]:
# resort
for seed, epi_data in data.items():
    for obs_index, obs_ret in epi_data.items():
        if obs_index == "seed":
            continue
        resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
        resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
        epi_data[obs_index] = resort


In [162]:
# average
averaged_ret = {}
for seed, epi_data in data.items():
    for obs, obs_ret in epi_data.items():
        if obs == "seed":
            continue
        if obs in averaged_ret:
            for k,each_neuron in enumerate(obs_ret):
                assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["seed_num"]+=1
        else:
            averaged_ret[obs] = []
            for each_neuron in obs_ret:
                averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                   "neuron_index": each_neuron["neuron"]["neuron_index"],
                                   "freq_diff": each_neuron["error"]["freq_diff"],
                                   "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                   "seed_num": 1})
for obs_dim, avg_ret in averaged_ret.items():
    averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)

rets[path] = averaged_ret

In [163]:
for key, value in averaged_ret.items():
    remove = []
    for k,unit in enumerate(value):
        if unit["neuron_index"]>=128 and unit["layer"]==0:
            remove.append(unit)
        elif unit["neuron_index"]>=64 and unit["layer"]==1:
            remove.append(unit)
        elif unit["neuron_index"]>=32 and unit["layer"]==2:
            remove.append(unit)
    for k in remove:
        value.remove(k)

In [164]:
print(averaged_ret[6])

[{'layer': 0, 'neuron_index': 66, 'freq_diff': 32.582218908879575, 'base_phase_diff': 0.896640579911691, 'seed_num': 10}, {'layer': 0, 'neuron_index': 14, 'freq_diff': 34.197117874749324, 'base_phase_diff': 0.8124880421946628, 'seed_num': 10}, {'layer': 0, 'neuron_index': 20, 'freq_diff': 34.76637090301961, 'base_phase_diff': 0.8940101734723793, 'seed_num': 10}, {'layer': 0, 'neuron_index': 67, 'freq_diff': 36.23504358182592, 'base_phase_diff': 0.5428692241985849, 'seed_num': 10}, {'layer': 0, 'neuron_index': 126, 'freq_diff': 36.922417863356486, 'base_phase_diff': 0.8451353031520015, 'seed_num': 10}, {'layer': 0, 'neuron_index': 113, 'freq_diff': 41.792724239158666, 'base_phase_diff': 0.8257303483399365, 'seed_num': 10}, {'layer': 0, 'neuron_index': 31, 'freq_diff': 44.51390355072298, 'base_phase_diff': 0.759812514411794, 'seed_num': 10}, {'layer': 0, 'neuron_index': 11, 'freq_diff': 44.875806242591054, 'base_phase_diff': 0.7064700549696281, 'seed_num': 10}, {'layer': 0, 'neuron_index

In [ ]:

# 12-23  +1
# 24-35 +1 497 230 40
print(rets["cassie_0_z_pd_ret"][0])